In [1]:
import config
import pandas as pd
import psycopg2
import numpy as np

Code acte consultation : « CNP »
Base tarif : 42,5 (Secteur 2 non OPTAM)
MPC : majoration du médecin spécialiste : 2,7 euros
Donc S1 et S2 OPTAM = 45,2
keep if strpos(v17,"CNP")
MCS: MCS : majoration de coordination pour les psychiatres : 5 euros

In [2]:
datesource_min = 2301
datesource_max = 2308
cs = 42.5
mpc = 2.7
mcs = 5
tarif_s1 = cs + mpc + mcs
tarif_s1

50.2

In [3]:
sql = f"""
select * from ps p
join tarif t on t.ps_id =p.id
join tarif_date_source tds on tds.tarif_id =t.id 
join cabinet c on t.cabinet_id = c.id
join adresse_raw ar on c.adresse_raw_id = ar.id
join adresse_norm an on ar.adresse_norm_id = an.id
join ban b on an.ban_id  = b.id
where (t.profession_id =65 or t.profession_id =66)  # Faire mune jointure avec les diplomes vs profession j'ai l'info psy = 65 or 66 qqpart
and tds.date_source_id >= {datesource_min} and  tds.date_source_id <= {datesource_max}
"""

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.expand_frame_repr', False)

# df = pd.read_sql(sql, config.connection_string)
df = pd.read_csv("C:/Users/conta/git-CVC/Skema/Part III/psychiatres.csv", low_memory=False)
df.describe()


,id,rule_nb,id.1,profession_id,mode_exercice_id,nature_id,convention_id,ps_id,cabinet_id,famille_acte_id,montant,borne_inf,borne_sup,montant_2,borne_inf_2,borne_sup_2,montant_imagerie,borne_inf_imagerie,borne_sup_imagerie,montant_anesthesie,borne_inf_anesthesie,borne_sup_anesthesie,montant_cec,borne_inf_cec,borne_sup_cec,tarif_id,date_source_id,id.2,telephone,adresse_raw_id,id.3,cp,adresse_norm_id,dept_id,id.4,numero,cp.1,osm_id,osm_score,ban_id,ban_score,source_id,lon,lat,score,dept_id.1,id.5,numero.1,code_postal,lon.1,lat.1,dept_id.2
count,62173.000000,60775.000000,6.217300e+04,62173.000000,0.0,62173.000000,62173.00000,62173.000000,62173.000000,12361.000000,62173.000000,62173.000000,62173.000000,0.0,0.0,0.0,12361.0,12361.0,12361.0,12361.000000,12361.000000,12361.000000,12361.0,12361.0,12361.0,6.217300e+04,62173.000000,62173.000000,4.659200e+04,62173.000000,62173.000000,62173.000000,62173.000000,62173.000000,62173.000000,58793.000000,62173.000000,4805.000000,4869.000000,6.217300e+04,62173.000000,62173.000000,62173.000000,62173.000000,62173.000000,62173.000000,6.217300e+04,61852.000000,62173.000000,62173.000000,62173.000000,62173.000000
mean,728301.521722,1.729165,2.002598e+07,65.013398,NaN,2.944108,1.89994,728301.521722,96303.205089,183.171426,35.301258,25.998795,34.364523,NaN,NaN,NaN,0.0,0.0,0.0,1.627053,0.570828,0.881887,0.0,0.0,0.0,2.002598e+07,2304.473823,96303.205089,4.343638e+08,77076.528573,77076.528573,52980.329484,77619.717273,53.948804,77619.717273,62.395846,52980.329484,14343.288866,0.812670,1.335062e+07,0.980543,2.092934,2.856080,46.675887,0.985698,53.948804,1.335062e+07,61.668402,52978.922812,2.856655,46.669992,53.948804
std,44495.514880,2.053991,2.606670e+06,0.114973,NaN,0.565625,0.99486,44495.514880,59623.065558,36.148945,44.621316,35.197314,46.984905,NaN,NaN,NaN,0.0,0.0,0.0,8.686618,5.203467,8.103884,0.0,0.0,0.0,2.606670e+06,2.294107,59623.065558,2.130620e+08,84888.944104,84888.944104,26293.534649,82095.580287,28.701967,82095.580287,164.191753,26293.534649,6978.001223,0.166871,7.277875e+06,0.045018,0.434307,2.671546,2.469264,0.033940,28.701967,7.277875e+06,179.944063,26292.988547,2.671045,2.525103,28.701967
min,648102.000000,1.000000,1.584330e+07,65.000000,NaN,1.000000,1.00000,648102.000000,63.000000,13.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.584330e+07,2301.000000,63.000000,1.300762e+08,60.000000,60.000000,1000.000000,60.000000,1.000000,60.000000,1.000000,1000.000000,6348.000000,0.000000,2.983000e+03,0.671429,1.000000,-4.530457,0.000000,0.571429,1.000000,2.983000e+03,1.000000,1000.000000,-4.530457,0.000000,1.000000
25%,693181.000000,1.000000,1.751675e+07,65.000000,NaN,3.000000,1.00000,693181.000000,50489.000000,201.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.751675e+07,2302.000000,50489.000000,2.412315e+08,25753.000000,25753.000000,33000.000000,26729.000000,33.000000,26729.000000,7.000000,33000.000000,9017.000000,0.700000,7.292780e+06,0.987500,2.000000,1.445443,44.015019,0.987500,33.000000,7.292780e+06,6.000000,33000.000000,1.445443,44.013227,33.000000
50%,730155.000000,1.000000,1.996902e+07,65.000000,NaN,3.000000,1.00000,730155.000000,91943.000000,201.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.996902e+07,2304.000000,91943.000000,4.685052e+08,48614.000000,48614.000000,59260.000000,50162.000000,59.000000,50162.000000,20.000000,59260.000000,12648.000000,0.850000,1.474418e+07,1.000000,2.000000,2.381545,47.249092,1.000000,59.000000,1.474418e+07,18.000000,59260.000000,2.381523,47.245400,59.000000
75%,768750.000000,1.000000,2.264998e+07,65.000000,NaN,3.000000,3.00000,768750.000000,135649.000000,201.000000,70.000000,50.200000,75.000000,NaN,NaN,NaN,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,2.264998e+07,2306.000000,135649.000000,5.611559e+08,80132.000000,80132.000000,75012.000000,85014.000000,75.000000,85014.000000,55.000000,75012.000000,1522

In [4]:
# Renommer les colonnes
df.rename(columns={
    'genre': 'gender',
    'prenom': 'prénom',
    'code': 'codeccamdelacte',
    'nature_id': 'naturedexercice',
    'convention_id': 'convention',
    'code_insee': 'codeinsee',
    'code_postal': 'matchcp',
    'montant': 'montantgénéralementconstaté',
    'borne_inf': 'borneinférieuredumontant',
    'borne_sup': 'bornesupérieuredumontant',
    'option_contrat': 'optioncontratdaccèsauxsoins',
    'dept_id': 'dep'
}, inplace=True)

# Colonnes à conserver
df = df[['ps_id', 'gender', 'nom', 'prénom', 'codeccamdelacte', 'naturedexercice', 'convention',
         'codeinsee', 'montantgénéralementconstaté', 'borneinférieuredumontant', 'bornesupérieuredumontant',
         'adresse_id', 'date_source_id', 'optioncontratdaccèsauxsoins', 'matchcp', 'dep']]
df

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep
0,649908,F,BAUDOIN,HELENE,CNP+MPC+MCS,3,1,6088,0.0,0.0,0.0,06088_4790_00012,2301,False,6000,6
1,649908,F,BAUDOIN,HELENE,CNP+MPC+MCS,3,1,6088,0.0,0.0,0.0,06088_4790_00012,2302,False,6000,6
2,649908,F,BAUDOIN,HELENE,CNP+MPC+MCS,3,1,6088,0.0,0.0,0.0,06088_4790_00012,2303,False,6000,6
3,649908,F,BAUDOIN,HELENE,CNP+MPC+MCS,3,1,6088,0.0,0.0,0.0,06088_4790_00012,2304,False,6000,6
4,649908,F,BAUDOIN,HELENE,CNP+MPC+MCS,3,1,6088,0.0,0.0,0.0,06088_4790_00012,2305,False,6000,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62168,729579,H,AKHRIF,MEHDI,CNP+MPC+MCS,2,1,45155,0.0,0.0,0.0,45155_0412_00001,2304,False,45500,45
62169,729579,H,AKHRIF,MEHDI,CNP+MPC+MCS,2,1,45155,0.0,0.0,0.0,45155_0412_00001,2305,False,45500,45
62170,729579,H,AKHRIF,MEHDI,CNP+MPC+MCS,2,1,45155,0.0,0.0,0.0,45155_0412_00001,2306,False,45500,45
62171,729579,H,AKHRIF,MEHDI,CNP+MPC+MCS,2,1,45155,0.0,0.0,0.0,45155_0412_00001,2307,False,45500,45


In [5]:
# Clé combinée
df['b'] = df['ps_id'].astype(str) + df['adresse_id'].astype(str)
df

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b
0,649908,F,BAUDOIN,HELENE,CNP+MPC+MCS,3,1,6088,0.0,0.0,0.0,06088_4790_00012,2301,False,6000,6,64990806088_4790_00012
1,649908,F,BAUDOIN,HELENE,CNP+MPC+MCS,3,1,6088,0.0,0.0,0.0,06088_4790_00012,2302,False,6000,6,64990806088_4790_00012
2,649908,F,BAUDOIN,HELENE,CNP+MPC+MCS,3,1,6088,0.0,0.0,0.0,06088_4790_00012,2303,False,6000,6,64990806088_4790_00012
3,649908,F,BAUDOIN,HELENE,CNP+MPC+MCS,3,1,6088,0.0,0.0,0.0,06088_4790_00012,2304,False,6000,6,64990806088_4790_00012
4,649908,F,BAUDOIN,HELENE,CNP+MPC+MCS,3,1,6088,0.0,0.0,0.0,06088_4790_00012,2305,False,6000,6,64990806088_4790_00012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62168,729579,H,AKHRIF,MEHDI,CNP+MPC+MCS,2,1,45155,0.0,0.0,0.0,45155_0412_00001,2304,False,45500,45,72957945155_0412_00001
62169,729579,H,AKHRIF,MEHDI,CNP+MPC+MCS,2,1,45155,0.0,0.0,0.0,45155_0412_00001,2305,False,45500,45,72957945155_0412_00001
62170,729579,H,AKHRIF,MEHDI,CNP+MPC+MCS,2,1,45155,0.0,0.0,0.0,45155_0412_00001,2306,False,45500,45,72957945155_0412_00001
62171,729579,H,AKHRIF,MEHDI,CNP+MPC+MCS,2,1,45155,0.0,0.0,0.0,45155_0412_00001,2307,False,45500,45,72957945155_0412_00001


In [6]:
nb_ps_id = df["ps_id"].nunique()
nb_ps_id  # 5439


5439

In [7]:
nb_b = df["b"].nunique()
nb_b  # 5674

5674

In [8]:

df = df.sort_values(by='ps_id')
df


,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b
31684,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2302,False,1200,1,64810201091_0331_00539
31686,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2304,False,1200,1,64810201091_0331_00539
31687,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2305,False,1200,1,64810201091_0331_00539
31688,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2306,False,1200,1,64810201091_0331_00539
31689,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2307,False,1200,1,64810201091_0331_00539
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61230,799634,F,VINCENTI,ODILE,CNP+MPC+MCS,3,1,13209,0.0,0.0,0.0,13209_4629_00002,2303,False,13009,13,79963413209_4629_00002
61235,799634,F,VINCENTI,ODILE,CNP+MPC+MCS,3,1,13209,0.0,0.0,0.0,13209_4629_00002,2308,False,13009,13,79963413209_4629_00002
15325,799731,F,GAUDIN,MAUD,CNP+MPC+MCS,3,1,31188,0.0,0.0,0.0,31188_2noilo_00003,2308,False,31470,31,79973131188_2noilo_00003
15324,799731,F,GAUDIN,MAUD,CNP+MPC+MCS,3,1,31188,0.0,0.0,0.0,31188_2noilo_00003,2307,False,31470,31,79973131188_2noilo_00003


In [9]:
df['unique'] = df.groupby('ps_id')['adresse_id'].transform('nunique')
df['un'] = df['unique']
pd.set_option('display.max_rows', 200)
df.head(100)

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un
31684,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2302,False,1200,1,64810201091_0331_00539,1,1
31686,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2304,False,1200,1,64810201091_0331_00539,1,1
31687,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2305,False,1200,1,64810201091_0331_00539,1,1
31688,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2306,False,1200,1,64810201091_0331_00539,1,1
31689,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2307,False,1200,1,64810201091_0331_00539,1,1
31690,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2308,False,1200,1,64810201091_0331_00539,1,1
31683,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2301,False,1200,1,64810201091_0331_00539,1,1
31685,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2303,False,1200,1,64810201091_0331_00539,1,1
31715,648107,F,SPIRIDONOVA,LUBA,ALQP0030,3,1,1033,0.0,0.0,0.0,01091_0331_00539,2301,False,1200,1,64810701091_0331_00539,1,1
31716,648107,F,SPIRIDONOVA,LUBA,ALQP0030,3,1,1033,0.0,0.0,0.0,01091_0331_00539,2302,False,1200,1,64810701091_0331_00539,1,1


In [10]:
# Comptages globaux
df['NB_total'] = df.groupby('dep')['ps_id'].transform('nunique')
df = df.sort_values(by='dep')
df["NB_total"].unique()

array([  14,   24,   10,   11,    6,  188,    8,    7,    5,   15,  346,
         40,    3,   34,   45,   28,   17,   38,   20,   26,    9,   56,
         53,  204,  289,  164,   86,   66,  118,   22,   49,  113,    2,
         58,   35,    4,   64,   41,   44,  122,   19,   32,   81,   72,
         12,   33,  145,  272,   57, 1016,   76,  102,   13,   89,   39,
         18,   29,  132,   52], dtype=int64)

In [11]:
mask_s2 = (df['convention'].isin([2, 3])) | ((df['convention'] == 1) & (df['optioncontratdaccèsauxsoins']))
df['NB2_total'] = df[mask_s2].groupby('dep')['ps_id'].transform('nunique')
df['NB2_total'] = df.groupby('dep')['NB2_total'].transform('max')
df["NB2_total"] = df["NB2_total"].fillna(0)
df["NB2_total"].unique()



array([  6.,   3.,  82.,   2.,   1.,   0., 134.,  16.,   5.,  13.,   7.,
         4.,  11.,  44., 116.,  39.,  23.,  28.,  38.,  18.,  57.,  25.,
        41.,  10.,  15.,  72.,  26.,   8.,  84.,  27., 179.,  35., 700.,
        31.,  65.,  33.,  87.,  14.,  43.,   9.])

In [12]:
df['c'] = 1
df['NB_Ftotal'] = df.groupby('c')['ps_id'].transform('nunique')
df["NB_Ftotal"].unique()

array([5439], dtype=int64)

In [13]:
df['NB2_Ftotal'] = df[mask_s2].groupby('c')['ps_id'].transform('nunique')
df['NB2_Ftotal'] = df['NB2_Ftotal'].max()
df["NB2_Ftotal"].unique()

array([2436.])

In [14]:
# Pondération
df['weight'] = df['un'].map({1: 1, 2: 0.5, 3: 0.33, 4: 0.25, 5: 0.2}).fillna(0)
df

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,weight
31684,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2302,False,1200,1,64810201091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0
60728,739108,F,CHIRON,EMMANUELLE,CNP+MPC+MCS,3,1,1322,0.0,0.0,0.0,01322_0240_00127,2308,False,1600,1,73910801322_0240_00127,1,1,14,6.0,1,5439,2436.0,1.0
60723,739108,F,CHIRON,EMMANUELLE,CNP+MPC+MCS,3,1,1322,0.0,0.0,0.0,01322_0240_00127,2303,False,1600,1,73910801322_0240_00127,1,1,14,6.0,1,5439,2436.0,1.0
57130,697644,H,SERMET,ERIC,ALQP0030,3,1,1033,69.1,0.0,0.0,01091_0331_00539,2306,True,1200,1,69764401091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0
57131,697644,H,SERMET,ERIC,ALQP0030,3,1,1033,69.1,0.0,0.0,01091_0331_00539,2307,True,1200,1,69764401091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2700,659292,H,STALLA,PATRICK,CNP+MPC+MCS,3,1,2B042,0.0,0.0,0.0,2b042_0293_00005,2306,False,20290,202,6592922b042_0293_00005,1,1,15,9.0,1,5439,2436.0,1.0
2702,659292,H,STALLA,PATRICK,CNP+MPC+MCS,3,1,2B042,0.0,0.0,0.0,2b042_0293_00005,2308,False,20290,202,6592922b042_0293_00005,1,1,15,9.0,1,5439,2436.0,1.0
2701,659292,H,STALLA,PATRICK,CNP+MPC+MCS,3,1,2B042,0.0,0.0,0.0,2b042_0293_00005,2307,False,20290,202,6592922b042_0293_00005,1,1,15,9.0,1,5439,2436.0,1.0
2716,659339,H,BALDOCCHI,DUMENICU,AZRP0010,3,3,2B042,39.2,0.0,0.0,2b042_0293_00005,2305,True,20290,202,6593392b042_0293_00005,2,2,15,9.0,1,5439,2436.0,0.5


In [15]:
# Tous les praticiens ne font pas de consultation
nb_ps_id, df["ps_id"].nunique()

(5439, 5439)

In [16]:
# Filtrage sur les actes de consultation
df = df[df['codeccamdelacte'].str.contains("CNP")]
# 45586
df

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,weight
31684,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2302,False,1200,1,64810201091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0
60728,739108,F,CHIRON,EMMANUELLE,CNP+MPC+MCS,3,1,1322,0.0,0.0,0.0,01322_0240_00127,2308,False,1600,1,73910801322_0240_00127,1,1,14,6.0,1,5439,2436.0,1.0
60723,739108,F,CHIRON,EMMANUELLE,CNP+MPC+MCS,3,1,1322,0.0,0.0,0.0,01322_0240_00127,2303,False,1600,1,73910801322_0240_00127,1,1,14,6.0,1,5439,2436.0,1.0
57133,697644,H,SERMET,ERIC,CNP+MPC+MCS,3,1,1033,50.2,50.2,50.2,01091_0331_00539,2301,True,1200,1,69764401091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0
57134,697644,H,SERMET,ERIC,CNP+MPC+MCS,3,1,1033,50.2,50.2,50.2,01091_0331_00539,2302,True,1200,1,69764401091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2698,659292,H,STALLA,PATRICK,CNP+MPC+MCS,3,1,2B042,0.0,0.0,0.0,2b042_0293_00005,2304,False,20290,202,6592922b042_0293_00005,1,1,15,9.0,1,5439,2436.0,1.0
2700,659292,H,STALLA,PATRICK,CNP+MPC+MCS,3,1,2B042,0.0,0.0,0.0,2b042_0293_00005,2306,False,20290,202,6592922b042_0293_00005,1,1,15,9.0,1,5439,2436.0,1.0
2702,659292,H,STALLA,PATRICK,CNP+MPC+MCS,3,1,2B042,0.0,0.0,0.0,2b042_0293_00005,2308,False,20290,202,6592922b042_0293_00005,1,1,15,9.0,1,5439,2436.0,1.0
2701,659292,H,STALLA,PATRICK,CNP+MPC+MCS,3,1,2B042,0.0,0.0,0.0,2b042_0293_00005,2307,False,20290,202,6592922b042_0293_00005,1,1,15,9.0,1,5439,2436.0,1.0


In [17]:
df["codeccamdelacte"].value_counts(normalize=True)

codeccamdelacte
CNP+MPC+MCS        0.741851
CNP                0.252731
CNP+MP             0.003137
CNP+MPC+MCS+MTX    0.001930
CNP+MPC+MTX        0.000351
Name: proportion, dtype: float64

In [18]:
df = df.sort_values(by='convention')
df

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,weight
31684,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2302,False,1200,1,64810201091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0
56330,756003,H,FONTVIEILLE,CHRISTIAN,CNP+MPC+MCS,2,1,63113,0.0,0.0,0.0,63113_3725_00018,2302,False,63000,63,75600363113_3725_00018,1,1,81,10.0,1,5439,2436.0,1.0
56329,756003,H,FONTVIEILLE,CHRISTIAN,CNP+MPC+MCS,2,1,63113,0.0,0.0,0.0,63113_3725_00018,2301,False,63000,63,75600363113_3725_00018,1,1,81,10.0,1,5439,2436.0,1.0
56336,756003,H,FONTVIEILLE,CHRISTIAN,CNP+MPC+MCS,2,1,63113,0.0,0.0,0.0,63113_3725_00018,2308,False,63000,63,75600363113_3725_00018,1,1,81,10.0,1,5439,2436.0,1.0
13582,756042,H,ENJOLRAS,JULIEN,CNP+MPC+MCS,3,1,63113,0.0,0.0,0.0,63113_0450_00009,2301,False,63000,63,75604263113_0450_00009,1,1,81,10.0,1,5439,2436.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26478,765534,F,BERNASCONI HOVAERE,SOPHIE,CNP+MPC+MCS,3,3,26191,50.2,50.2,50.2,26191_f740ze_00290_a,2305,False,26740,26,76553426191_f740ze_00290_a,1,1,20,6.0,1,5439,2436.0,1.0
26477,765534,F,BERNASCONI HOVAERE,SOPHIE,CNP+MPC+MCS,3,3,26191,50.2,50.2,50.2,26191_f740ze_00290_a,2304,False,26740,26,76553426191_f740ze_00290_a,1,1,20,6.0,1,5439,2436.0,1.0
26474,765534,F,BERNASCONI HOVAERE,SOPHIE,CNP+MPC+MCS,3,3,26191,50.2,50.2,50.2,26191_f740ze_00290_a,2301,False,26740,26,76553426191_f740ze_00290_a,1,1,20,6.0,1,5439,2436.0,1.0
37737,770905,F,GRADEL VROUST,IRIS,CNP,2,3,69089,85.0,65.0,88.5,69089_0540_00003,2304,False,69340,69,77090569089_0540_00003,1,1,272,179.0,1,5439,2436.0,1.0


In [19]:
df.groupby('convention')["ps_id"].nunique()

convention
1    3072
2       2
3    2337
Name: ps_id, dtype: int64

In [20]:
df = df.sort_values(by=['b',"convention", "codeccamdelacte", "date_source_id"])
df.head(20)

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,weight
31683,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2301,False,1200,1,64810201091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0
31684,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2302,False,1200,1,64810201091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0
31685,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2303,False,1200,1,64810201091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0
31686,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2304,False,1200,1,64810201091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0
31687,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2305,False,1200,1,64810201091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0
31688,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2306,False,1200,1,64810201091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0
31689,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2307,False,1200,1,64810201091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0
31690,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2308,False,1200,1,64810201091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0
31723,648107,F,SPIRIDONOVA,LUBA,CNP+MPC+MCS,3,1,1033,0.0,0.0,0.0,01091_0331_00539,2301,False,1200,1,64810701091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0
31724,648107,F,SPIRIDONOVA,LUBA,CNP+MPC+MCS,3,1,1033,0.0,0.0,0.0,01091_0331_00539,2302,False,1200,1,64810701091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0


In [21]:
# Moyenne par groupe
df['mp'] = df.groupby(['b', 'convention', 'codeccamdelacte'])['montantgénéralementconstaté'].transform('mean')
df.head(30)

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,weight,mp
31683,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2301,False,1200,1,64810201091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,83.5
31684,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2302,False,1200,1,64810201091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,83.5
31685,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2303,False,1200,1,64810201091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,83.5
31686,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2304,False,1200,1,64810201091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,83.5
31687,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2305,False,1200,1,64810201091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,83.5
31688,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2306,False,1200,1,64810201091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,83.5
31689,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2307,False,1200,1,64810201091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,83.5
31690,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2308,False,1200,1,64810201091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,83.5
31723,648107,F,SPIRIDONOVA,LUBA,CNP+MPC+MCS,3,1,1033,0.0,0.0,0.0,01091_0331_00539,2301,False,1200,1,64810701091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,0.0
31724,648107,F,SPIRIDONOVA,LUBA,CNP+MPC+MCS,3,1,1033,0.0,0.0,0.0,01091_0331_00539,2302,False,1200,1,64810701091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,0.0


In [22]:
# 45339
df = df[(df['codeccamdelacte'] == "CNP") | (df['codeccamdelacte'] == "CNP+MPC+MCS")]
df

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,weight,mp
31683,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2301,False,1200,1,64810201091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,83.5
31684,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2302,False,1200,1,64810201091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,83.5
31685,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2303,False,1200,1,64810201091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,83.5
31686,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2304,False,1200,1,64810201091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,83.5
31687,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2305,False,1200,1,64810201091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,83.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61234,799634,F,VINCENTI,ODILE,CNP+MPC+MCS,3,1,13209,0.0,0.0,0.0,13209_4629_00002,2307,False,13009,13,79963413209_4629_00002,1,1,346,134.0,1,5439,2436.0,1.0,0.0
61235,799634,F,VINCENTI,ODILE,CNP+MPC+MCS,3,1,13209,0.0,0.0,0.0,13209_4629_00002,2308,False,13009,13,79963413209_4629_00002,1,1,346,134.0,1,5439,2436.0,1.0,0.0
15324,799731,F,GAUDIN,MAUD,CNP+MPC+MCS,3,1,31188,0.0,0.0,0.0,31188_2noilo_00003,2307,False,31470,31,79973131188_2noilo_00003,1,1,204,44.0,1,5439,2436.0,1.0,0.0
15325,799731,F,GAUDIN,MAUD,CNP+MPC+MCS,3,1,31188,0.0,0.0,0.0,31188_2noilo_00003,2308,False,31470,31,79973131188_2noilo_00003,1,1,204,44.0,1,5439,2436.0,1.0,0.0


In [23]:
tarif_s1

50.2

In [24]:
# 23421
print(len(df[(df['convention'] == 1) & (df['optioncontratdaccèsauxsoins'] == False)]))
df.loc[(df['convention'] == 1) & (df['optioncontratdaccèsauxsoins'] == False), 'mp'] = tarif_s1
df


23421


,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,weight,mp
31683,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2301,False,1200,1,64810201091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,50.2
31684,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2302,False,1200,1,64810201091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,50.2
31685,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2303,False,1200,1,64810201091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,50.2
31686,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2304,False,1200,1,64810201091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,50.2
31687,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2305,False,1200,1,64810201091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,50.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61234,799634,F,VINCENTI,ODILE,CNP+MPC+MCS,3,1,13209,0.0,0.0,0.0,13209_4629_00002,2307,False,13009,13,79963413209_4629_00002,1,1,346,134.0,1,5439,2436.0,1.0,50.2
61235,799634,F,VINCENTI,ODILE,CNP+MPC+MCS,3,1,13209,0.0,0.0,0.0,13209_4629_00002,2308,False,13009,13,79963413209_4629_00002,1,1,346,134.0,1,5439,2436.0,1.0,50.2
15324,799731,F,GAUDIN,MAUD,CNP+MPC+MCS,3,1,31188,0.0,0.0,0.0,31188_2noilo_00003,2307,False,31470,31,79973131188_2noilo_00003,1,1,204,44.0,1,5439,2436.0,1.0,50.2
15325,799731,F,GAUDIN,MAUD,CNP+MPC+MCS,3,1,31188,0.0,0.0,0.0,31188_2noilo_00003,2308,False,31470,31,79973131188_2noilo_00003,1,1,204,44.0,1,5439,2436.0,1.0,50.2


****on élimine les duplications ps-adresse convention acte puis on sélectionne le prix le plus élevé d'un acte de consultation (discard tarif opposable des secteurs 2)
***puis on élimine les duplications ps-adresse convention (car les actes ont le même prix maintenant pour chaque ps-adresse)

In [25]:
# Efface 39500
# Reste 5839
df = df.drop_duplicates(subset=['b', 'convention', 'codeccamdelacte'])
df

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,weight,mp
31683,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2301,False,1200,1,64810201091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,50.2
31723,648107,F,SPIRIDONOVA,LUBA,CNP+MPC+MCS,3,1,1033,0.0,0.0,0.0,01091_0331_00539,2301,False,1200,1,64810701091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,50.2
31675,648127,H,STENGER,STEPHANE,CNP+MPC+MCS,3,1,1266,0.0,0.0,0.0,01266_0065_00326,2301,False,1340,1,64812701266_0065_00326,1,1,14,6.0,1,5439,2436.0,1.0,50.2
27248,648312,F,MAUSSAC,MICHELE,CNP+MPC+MCS,3,3,1004,50.2,50.2,75.0,01004_0420_00209,2301,True,1500,1,64831201004_0420_00209,2,2,14,6.0,1,5439,2436.0,0.5,50.2
60553,648312,F,MAUSSAC,MICHELE,CNP+MPC+MCS,3,3,69383,50.2,50.2,85.0,69383_5220_00078,2301,True,69003,69,64831269383_5220_00078,2,2,272,179.0,1,5439,2436.0,0.5,50.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30805,799569,F,MICHAUT,CAROLINE,CNP+MPC+MCS,3,3,59009,50.2,50.2,50.2,59009_6050_00001,2301,True,59650,59,79956959009_6050_00001,1,1,122,72.0,1,5439,2436.0,1.0,50.2
61171,799615,F,RUBINSTENN,ANNE,CNP+MPC+MCS,3,1,31555,0.0,0.0,0.0,31555_6020_00005,2302,False,31400,31,79961531555_6020_00005,1,1,204,44.0,1,5439,2436.0,1.0,50.2
61230,799634,F,VINCENTI,ODILE,CNP+MPC+MCS,3,1,13209,0.0,0.0,0.0,13209_4629_00002,2303,False,13009,13,79963413209_4629_00002,1,1,346,134.0,1,5439,2436.0,1.0,50.2
15324,799731,F,GAUDIN,MAUD,CNP+MPC+MCS,3,1,31188,0.0,0.0,0.0,31188_2noilo_00003,2307,False,31470,31,79973131188_2noilo_00003,1,1,204,44.0,1,5439,2436.0,1.0,50.2


In [26]:
df = df.sort_values(by=['b',"convention"])
df.head(10)

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,weight,mp
31683,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2301,False,1200,1,64810201091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.00,50.2
31723,648107,F,SPIRIDONOVA,LUBA,CNP+MPC+MCS,3,1,1033,0.0,0.0,0.0,01091_0331_00539,2301,False,1200,1,64810701091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.00,50.2
31675,648127,H,STENGER,STEPHANE,CNP+MPC+MCS,3,1,1266,0.0,0.0,0.0,01266_0065_00326,2301,False,1340,1,64812701266_0065_00326,1,1,14,6.0,1,5439,2436.0,1.00,50.2
27248,648312,F,MAUSSAC,MICHELE,CNP+MPC+MCS,3,3,1004,50.2,50.2,75.0,01004_0420_00209,2301,True,1500,1,64831201004_0420_00209,2,2,14,6.0,1,5439,2436.0,0.50,50.2
60553,648312,F,MAUSSAC,MICHELE,CNP+MPC+MCS,3,3,69383,50.2,50.2,85.0,69383_5220_00078,2301,True,69003,69,64831269383_5220_00078,2,2,272,179.0,1,5439,2436.0,0.50,50.2
62037,648449,H,HADDAG,NACER,CNP+MPC+MCS,3,1,2361,0.0,0.0,0.0,02361_0040_00041,2301,False,2120,2,64844902361_0040_00041,3,3,24,3.0,1,5439,2436.0,0.33,50.2
62069,648449,H,HADDAG,NACER,CNP+MPC+MCS,4,1,2691,0.0,0.0,0.0,02691_1080_00036,2301,False,2100,2,64844902691_1080_00036,3,3,24,3.0,1,5439,2436.0,0.33,50.2
31747,648503,H,KHARIJ,BRAHIM,CNP+MPC+MCS,3,1,2691,0.0,0.0,0.0,02691_1850_00009,2301,False,2100,2,64850302691_1850_00009,1,1,24,3.0,1,5439,2436.0,1.00,50.2
61991,648584,H,DELSAUX,JONATHAN,CNP+MPC+MCS,3,1,2691,0.0,0.0,0.0,02691_4370_00004,2301,False,2100,2,64858402691_4370_00004,1,1,24,3.0,1,5439,2436.0,1.00,50.2
62053,648597,H,LENDER,PIERRE ETIENNE,CNP+MPC+MCS,3,1,2691,0.0,0.0,0.0,02691_1080_00036,2301,False,2100,2,64859702691_1080_00036,1,1,24,3.0,1,5439,2436.0,1.00,50.2


In [27]:
df['prixmoyen'] = df.groupby(['b', 'convention'])['mp'].transform('max')
df.head(30)

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,weight,mp,prixmoyen
31683,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2301,False,1200,1,64810201091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.00,50.2,50.2
31723,648107,F,SPIRIDONOVA,LUBA,CNP+MPC+MCS,3,1,1033,0.0,0.0,0.0,01091_0331_00539,2301,False,1200,1,64810701091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.00,50.2,50.2
31675,648127,H,STENGER,STEPHANE,CNP+MPC+MCS,3,1,1266,0.0,0.0,0.0,01266_0065_00326,2301,False,1340,1,64812701266_0065_00326,1,1,14,6.0,1,5439,2436.0,1.00,50.2,50.2
27248,648312,F,MAUSSAC,MICHELE,CNP+MPC+MCS,3,3,1004,50.2,50.2,75.0,01004_0420_00209,2301,True,1500,1,64831201004_0420_00209,2,2,14,6.0,1,5439,2436.0,0.50,50.2,50.2
60553,648312,F,MAUSSAC,MICHELE,CNP+MPC+MCS,3,3,69383,50.2,50.2,85.0,69383_5220_00078,2301,True,69003,69,64831269383_5220_00078,2,2,272,179.0,1,5439,2436.0,0.50,50.2,50.2
62037,648449,H,HADDAG,NACER,CNP+MPC+MCS,3,1,2361,0.0,0.0,0.0,02361_0040_00041,2301,False,2120,2,64844902361_0040_00041,3,3,24,3.0,1,5439,2436.0,0.33,50.2,50.2
62069,648449,H,HADDAG,NACER,CNP+MPC+MCS,4,1,2691,0.0,0.0,0.0,02691_1080_00036,2301,False,2100,2,64844902691_1080_00036,3,3,24,3.0,1,5439,2436.0,0.33,50.2,50.2
31747,648503,H,KHARIJ,BRAHIM,CNP+MPC+MCS,3,1,2691,0.0,0.0,0.0,02691_1850_00009,2301,False,2100,2,64850302691_1850_00009,1,1,24,3.0,1,5439,2436.0,1.00,50.2,50.2
61991,648584,H,DELSAUX,JONATHAN,CNP+MPC+MCS,3,1,2691,0.0,0.0,0.0,02691_4370_00004,2301,False,2100,2,64858402691_4370_00004,1,1,24,3.0,1,5439,2436.0,1.00,50.2,50.2
62053,648597,H,LENDER,PIERRE ETIENNE,CNP+MPC+MCS,3,1,2691,0.0,0.0,0.0,02691_1080_00036,2301,False,2100,2,64859702691_1080_00036,1,1,24,3.0,1,5439,2436.0,1.00,50.2,50.2


In [28]:
# Reste 5522
df = df.drop_duplicates(subset=['b', 'convention'])
df

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,weight,mp,prixmoyen
31683,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2301,False,1200,1,64810201091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,50.2,50.2
31723,648107,F,SPIRIDONOVA,LUBA,CNP+MPC+MCS,3,1,1033,0.0,0.0,0.0,01091_0331_00539,2301,False,1200,1,64810701091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,50.2,50.2
31675,648127,H,STENGER,STEPHANE,CNP+MPC+MCS,3,1,1266,0.0,0.0,0.0,01266_0065_00326,2301,False,1340,1,64812701266_0065_00326,1,1,14,6.0,1,5439,2436.0,1.0,50.2,50.2
27248,648312,F,MAUSSAC,MICHELE,CNP+MPC+MCS,3,3,1004,50.2,50.2,75.0,01004_0420_00209,2301,True,1500,1,64831201004_0420_00209,2,2,14,6.0,1,5439,2436.0,0.5,50.2,50.2
60553,648312,F,MAUSSAC,MICHELE,CNP+MPC+MCS,3,3,69383,50.2,50.2,85.0,69383_5220_00078,2301,True,69003,69,64831269383_5220_00078,2,2,272,179.0,1,5439,2436.0,0.5,50.2,50.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30805,799569,F,MICHAUT,CAROLINE,CNP+MPC+MCS,3,3,59009,50.2,50.2,50.2,59009_6050_00001,2301,True,59650,59,79956959009_6050_00001,1,1,122,72.0,1,5439,2436.0,1.0,50.2,50.2
61171,799615,F,RUBINSTENN,ANNE,CNP+MPC+MCS,3,1,31555,0.0,0.0,0.0,31555_6020_00005,2302,False,31400,31,79961531555_6020_00005,1,1,204,44.0,1,5439,2436.0,1.0,50.2,50.2
61230,799634,F,VINCENTI,ODILE,CNP+MPC+MCS,3,1,13209,0.0,0.0,0.0,13209_4629_00002,2303,False,13009,13,79963413209_4629_00002,1,1,346,134.0,1,5439,2436.0,1.0,50.2,50.2
15324,799731,F,GAUDIN,MAUD,CNP+MPC+MCS,3,1,31188,0.0,0.0,0.0,31188_2noilo_00003,2307,False,31470,31,79973131188_2noilo_00003,1,1,204,44.0,1,5439,2436.0,1.0,50.2,50.2


In [29]:
df.groupby('convention')["ps_id"].nunique()

convention
1    3071
2       2
3    2337
Name: ps_id, dtype: int64

In [30]:
# df = df.drop_duplicates(subset=['b']) inutile
print(len(df[df["prixmoyen"] == 0])) #2
df['prixmoyen'] = df['prixmoyen'].replace(0, tarif_s1)
# ////Prix OK ////

2


In [36]:
# Calculs départementaux
df = df.sort_values(by="dep")
df['NB'] = df.groupby('dep')['ps_id'].transform('nunique')
df["NB"].unique() # Jamais == nan

array([  14,   24,   10,    6,  187,    7,    5,   15,   11,  341,   39,
          3,   34,   45,   28,   17,   38,   20,   26,    9,   56,   51,
        203,    8,  287,  164,   86,   66,  116,   22,   48,  112,    2,
         58,   35,    4,   64,   40,   44,  118,   19,   32,   81,   71,
         12,   33,  145,  270,   57, 1010,   76,  102,   13,   88,   18,
         21,   63,  131,   75], dtype=int64)

In [38]:
df['NB_F'] = df.groupby('c')['ps_id'].transform('nunique')
df["NB_F"].unique() # 5410

array([5410], dtype=int64)

In [54]:
mask_s2 = (df['convention'].isin([2, 3])) | ((df['convention'] == 1) & (df['optioncontratdaccèsauxsoins']))
df['NB2'] = df[mask_s2].groupby('dep')['ps_id'].transform('nunique')
df['NB2'] = df.groupby("dep")["NB2"].transform("mean")
df["NB2"] = df["NB2"].fillna(0)
df["NB2"].unique()
df[df["NB2"] == 0]["dep"].unique()

array([10, 12, 23, 32, 46, 48, 50, 53, 55, 58, 82, 88], dtype=int64)

In [56]:
df['NB2_F'] = df[mask_s2].groupby('c')['ps_id'].transform('nunique')
df['NB2_F'] = df['NB2_F'].mean()
df["NB2_F"].unique()  # 2418

array([2418.])

In [58]:
df['ShareS2'] = df['NB2'] / df['NB']
df['ShareS2_F'] = df['NB2_F'] / df['NB_F']
df.head(20)

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,weight,mp,prixmoyen,NB,NB_F,NB2,NB2_F,ShareS2,ShareS2_F
31683,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2301,False,1200,1,64810201091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.00,50.2,50.2,14,5410,6.0,2418.0,0.428571,0.44695
31675,648127,H,STENGER,STEPHANE,CNP+MPC+MCS,3,1,1266,0.0,0.0,0.0,01266_0065_00326,2301,False,1340,1,64812701266_0065_00326,1,1,14,6.0,1,5439,2436.0,1.00,50.2,50.2,14,5410,6.0,2418.0,0.428571,0.44695
27248,648312,F,MAUSSAC,MICHELE,CNP+MPC+MCS,3,3,1004,50.2,50.2,75.0,01004_0420_00209,2301,True,1500,1,64831201004_0420_00209,2,2,14,6.0,1,5439,2436.0,0.50,50.2,50.2,14,5410,6.0,2418.0,0.428571,0.44695
21541,753947,H,HOUOT,JEAN STEPHANE,CNP+MPC+MCS,2,3,1363,50.2,50.2,50.2,01363_0046_00180,2301,True,1640,1,75394701363_0046_00180,1,1,14,6.0,1,5439,2436.0,1.00,50.2,50.2,14,5410,6.0,2418.0,0.428571,0.44695
31731,742908,F,SELLIER,CLAIRE,CNP+MPC+MCS,3,1,1033,50.2,50.2,75.0,01033_0020_00009,2301,True,1200,1,74290801033_0020_00009,2,2,14,6.0,1,5439,2436.0,0.50,50.2,50.2,14,5410,6.0,2418.0,0.428571,0.44695
29422,788556,F,PRIOUX HABIB SABAU,CHERINE,CNP+MPC+MCS,3,1,1033,0.0,0.0,0.0,01091_0331_00539,2301,False,1200,1,78855601091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.00,50.2,50.2,14,5410,6.0,2418.0,0.428571,0.44695
61851,788555,F,VARICLIER,SOPHIE,CNP+MPC+MCS,3,3,1033,50.2,50.2,75.0,01091_0331_00539,2301,True,1200,1,78855501091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.00,50.2,50.2,14,5410,6.0,2418.0,0.428571,0.44695
31699,757518,H,MAURIN,ALAIN,CNP+MPC+MCS,3,3,1033,85.0,50.2,85.0,01091_0331_00539,2301,True,1200,1,75751801091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.00,85.0,85.0,14,5410,6.0,2418.0,0.428571,0.44695
61612,795001,F,JOHN,ALTHEA MARIA,CNP+MPC+MCS,3,1,1344,0.0,0.0,0.0,01344_0391_00039,2301,False,1000,1,79500101344_0391_00039,1,1,14,6.0,1,5439,2436.0,1.00,50.2,50.2,14,5410,6.0,2418.0,0.428571,0.44695
60721,739108,F,CHIRON,EMMANUELLE,CNP+MPC+MCS,3,1,1322,0.0,0.0,0.0,01322_0240_00127,2301,False,1600,1,73910801322_0240_00127,1,1,14,6.0,1,5439,2436.0,1.00,50.2,50.2,14,5410,6.0,2418.0,0.428571,0.44695
